# 岭回归 (Ridge Regression)

---

## 📚 学习目标

通过本教程，你将学会：
- 理解岭回归的原理和数学推导
- 掌握L2正则化如何防止过拟合
- 学会使用Scikit-learn实现岭回归
- 理解正则化参数α对模型的影响
- 比较岭回归与普通线性回归的区别

## 📋 前置知识

- 线性回归基础
- 多项式回归
- 过拟合与欠拟合概念

## ⏱️ 预计时间

25-35分钟

---

## 📖 第1部分：理论背景

### 什么是岭回归？

**岭回归（Ridge Regression）** 是一种带有**L2正则化**的线性回归模型。它通过在损失函数中添加正则化项来约束模型参数，从而防止过拟合。

### 为什么需要正则化？

在高维数据或存在多重共线性的情况下，普通最小二乘法（OLS）可能会：
- 产生非常大的参数值
- 对训练数据过度拟合
- 在测试数据上表现不佳

### 数学模型

#### 普通线性回归的损失函数

$$J(\theta) = \frac{1}{2m}\sum_{i=1}^{m}(h_\theta(x^{(i)}) - y^{(i)})^2 = \frac{1}{2m}||X\theta - y||^2$$

#### 岭回归的损失函数

$$J(\theta) = \frac{1}{2m}||X\theta - y||^2 + \alpha||\theta||^2$$

其中：
- $||\theta||^2 = \sum_{j=1}^{n}\theta_j^2$ 是L2范数的平方（不包括偏置项$\theta_0$）
- $\alpha$ 是正则化强度（超参数）

### 闭式解

岭回归的正规方程解为：

$$\hat{\theta} = (X^TX + \alpha I)^{-1}X^Ty$$

其中 $I$ 是单位矩阵（除了左上角为0，对应偏置项）。

### α参数的影响

| α值 | 效果 |
|-----|------|
| α = 0 | 退化为普通线性回归 |
| α 较小 | 轻微正则化 |
| α 较大 | 强正则化，参数趋向于0 |
| α → ∞ | 所有参数趋向于0（欠拟合）|

> 💡 **关键洞察**: 岭回归在损失函数中添加惩罚项，使得模型倾向于选择较小的参数值，从而降低模型复杂度，减少过拟合风险。

---

## 💻 第2部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

# 数值计算
import numpy as np

# 可视化
import matplotlib.pyplot as plt

# 机器学习
from sklearn.linear_model import Ridge, LinearRegression, SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

# ============================================================
# 配置参数
# ============================================================

# 设置随机种子，确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 数据生成参数
N_SAMPLES = 100
NOISE_LEVEL = 1.0

# 真实参数 (y = 0.5x² + x + 2)
TRUE_COEF_0 = 2.0    # 常数项
TRUE_COEF_1 = 1.0    # 一次项系数
TRUE_COEF_2 = 0.5    # 二次项系数

# 可视化配置
FIGURE_SIZE = (14, 5)
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 库导入和配置完成")

### 步骤2: 生成非线性数据

生成符合二次关系的数据，用于演示正则化效果：

In [ ]:
# ============================================================
# 生成非线性数据
# ============================================================

# 生成特征X：范围[-3, 3]
X = 6 * np.random.rand(N_SAMPLES, 1) - 3

# 生成目标y：y = 0.5x² + x + 2 + 噪声
y = TRUE_COEF_2 * X**2 + TRUE_COEF_1 * X + TRUE_COEF_0 + NOISE_LEVEL * np.random.randn(N_SAMPLES, 1)

# 为了演示过拟合，我们将使用一部分数据作为训练集
X_train, y_train = X[:20], y[:20]  # 使用较少的训练数据，容易过拟合
X_test, y_test = X[20:], y[20:]    # 测试数据

# 数据检查
print("✓ 数据生成完成")
print(f"  训练集大小: {len(X_train)}")
print(f"  测试集大小: {len(X_test)}")
print(f"  X范围: [{X.min():.2f}, {X.max():.2f}]")
print(f"  y范围: [{y.min():.2f}, {y.max():.2f}]")

### 步骤3: 基础岭回归实现

使用Scikit-learn的Ridge类实现基础岭回归：

In [ ]:
# ============================================================
# 基础岭回归
# ============================================================

# 创建岭回归模型
# alpha: 正则化强度，值越大正则化越强
# solver: 求解器，'cholesky'使用Cholesky分解求解
ridge_reg = Ridge(alpha=1.0, solver="cholesky")

# 训练模型
ridge_reg.fit(X, y)

# 预测新数据点
test_point = [[1.5]]
prediction = ridge_reg.predict(test_point)

print("基础岭回归结果:")
print(f"  模型参数: 截距={ridge_reg.intercept_[0]:.4f}, 系数={ridge_reg.coef_[0][0]:.4f}")
print(f"  预测 x=1.5 时: y = {prediction[0][0]:.4f}")
print(f"  真实值 (无噪声): y = {TRUE_COEF_2*1.5**2 + TRUE_COEF_1*1.5 + TRUE_COEF_0:.4f}")

### 步骤4: 使用SGD实现岭回归

对于大规模数据集，可以使用随机梯度下降（SGD）来实现岭回归：

In [ ]:
# ============================================================
# 使用SGD实现岭回归
# ============================================================

# SGDRegressor配合L2惩罚项等效于岭回归
# penalty="l2": 使用L2正则化
sgd_reg = SGDRegressor(
    penalty="l2",           # L2正则化 (岭回归)
    alpha=0.001,            # 正则化强度
    max_iter=1000,          # 最大迭代次数
    tol=1e-3,               # 收敛阈值
    random_state=RANDOM_SEED
)

# 训练模型（注意：y需要是1D数组）
sgd_reg.fit(X, y.ravel())

# 预测
sgd_prediction = sgd_reg.predict(test_point)

print("SGD岭回归结果:")
print(f"  模型参数: 截距={sgd_reg.intercept_[0]:.4f}, 系数={sgd_reg.coef_[0]:.4f}")
print(f"  预测 x=1.5 时: y = {sgd_prediction[0]:.4f}")
print(f"\n两种方法的预测差异: {abs(prediction[0][0] - sgd_prediction[0]):.6f}")

---

## 🔬 第3部分：正则化效果对比

### 不同α值的影响

让我们比较不同正则化强度对高次多项式拟合的影响：

In [ ]:
# ============================================================
# 不同α值对比实验
# ============================================================

# 使用高次多项式来演示过拟合和正则化效果
POLY_DEGREE = 10  # 使用10次多项式，容易过拟合

# 不同的正则化强度
alphas = [0, 0.1, 1, 10, 100]

# 生成用于绘图的平滑曲线
X_line = np.linspace(-3, 3, 100).reshape(-1, 1)

plt.figure(figsize=(16, 4))

for i, alpha in enumerate(alphas):
    plt.subplot(1, 5, i+1)
    
    # 创建管道：多项式特征 + 标准化 + 岭回归
    if alpha == 0:
        # α=0时使用普通线性回归
        model = Pipeline([
            ("poly_features", PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)),
            ("scaler", StandardScaler()),
            ("linear_reg", LinearRegression())
        ])
    else:
        model = Pipeline([
            ("poly_features", PolynomialFeatures(degree=POLY_DEGREE, include_bias=False)),
            ("scaler", StandardScaler()),
            ("ridge_reg", Ridge(alpha=alpha))
        ])
    
    # 使用少量训练数据训练模型
    model.fit(X_train, y_train)
    
    # 预测
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    y_line_pred = model.predict(X_line)
    
    # 计算MSE
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    
    # 绘图
    plt.scatter(X_train, y_train, color='blue', alpha=0.6, s=40, label='训练集')
    plt.scatter(X_test, y_test, color='green', alpha=0.3, s=20, label='测试集')
    plt.plot(X_line, y_line_pred, 'r-', linewidth=2)
    
    plt.xlabel('X')
    plt.ylabel('y')
    plt.title(f'α = {alpha}\n训练MSE: {train_mse:.2f}\n测试MSE: {test_mse:.2f}', fontsize=10)
    plt.ylim(-5, 15)
    plt.grid(True, alpha=0.3)
    
    if i == 0:
        plt.legend(fontsize=8)

plt.tight_layout()
plt.suptitle(f'{POLY_DEGREE}次多项式 - 不同正则化强度对比', y=1.02, fontsize=14, fontweight='bold')
plt.show()

print("\n观察结论:")
print("  - α=0 (无正则化): 严重过拟合，曲线波动剧烈")
print("  - α=0.1-1: 适度正则化，曲线平滑")
print("  - α=10-100: 强正则化，可能欠拟合")

### 参数收缩效果可视化

岭回归会将参数"收缩"向零，让我们可视化这个效果：

In [ ]:
# ============================================================
# 参数收缩路径图
# ============================================================

# 准备多项式特征
poly = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly.fit_transform(X)

# 标准化
scaler = StandardScaler()
X_poly_scaled = scaler.fit_transform(X_poly)

# 测试不同的alpha值
alphas_range = np.logspace(-2, 4, 50)  # 从0.01到10000
coefs = []

for alpha in alphas_range:
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_poly_scaled, y)
    coefs.append(ridge.coef_.flatten())

coefs = np.array(coefs)

# 绘制参数路径
plt.figure(figsize=(12, 5))

# 左图：参数收缩路径
plt.subplot(1, 2, 1)
for i in range(coefs.shape[1]):
    plt.plot(alphas_range, coefs[:, i], linewidth=1.5, label=f'θ{i+1}')

plt.xscale('log')
plt.xlabel('α (正则化强度)', fontsize=12)
plt.ylabel('参数值', fontsize=12)
plt.title('岭回归参数收缩路径', fontsize=14, fontweight='bold')
plt.legend(loc='upper right', fontsize=8, ncol=2)
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5)

# 右图：参数范数变化
plt.subplot(1, 2, 2)
coef_norms = np.linalg.norm(coefs, axis=1)
plt.plot(alphas_range, coef_norms, 'b-', linewidth=2)
plt.xscale('log')
plt.xlabel('α (正则化强度)', fontsize=12)
plt.ylabel('||θ||₂ (参数L2范数)', fontsize=12)
plt.title('参数范数随α变化', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n观察结论:")
print("  - 随着α增大，所有参数都被"收缩"向0")
print("  - 参数的L2范数单调递减")
print("  - 岭回归不会将参数精确收缩到0（与Lasso不同）")

---

## 📊 第4部分：交叉验证选择最佳α

使用交叉验证来选择最佳的正则化参数：

In [ ]:
# ============================================================
# 交叉验证选择最佳α
# ============================================================

from sklearn.linear_model import RidgeCV

# 候选的alpha值
alphas_cv = np.logspace(-3, 3, 100)

# 创建管道
cv_scores = []

for alpha in alphas_cv:
    model = Pipeline([
        ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
        ("scaler", StandardScaler()),
        ("ridge_reg", Ridge(alpha=alpha))
    ])
    
    # 5折交叉验证
    scores = cross_val_score(model, X, y.ravel(), 
                            scoring='neg_mean_squared_error', cv=5)
    cv_scores.append(-scores.mean())  # 转换为正的MSE

cv_scores = np.array(cv_scores)

# 找到最佳alpha
best_idx = np.argmin(cv_scores)
best_alpha = alphas_cv[best_idx]
best_score = cv_scores[best_idx]

# 可视化
plt.figure(figsize=(10, 5))
plt.plot(alphas_cv, cv_scores, 'b-', linewidth=2)
plt.axvline(x=best_alpha, color='r', linestyle='--', 
            label=f'最佳α = {best_alpha:.4f}')
plt.scatter([best_alpha], [best_score], color='red', s=100, zorder=5)
plt.xscale('log')
plt.xlabel('α (正则化强度)', fontsize=12)
plt.ylabel('交叉验证MSE', fontsize=12)
plt.title('交叉验证选择最佳正则化参数', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n最佳正则化参数: α = {best_alpha:.4f}")
print(f"对应的交叉验证MSE: {best_score:.4f}")

### 使用RidgeCV自动选择α

In [ ]:
# ============================================================
# 使用RidgeCV自动选择最佳alpha
# ============================================================

# 准备数据
poly = PolynomialFeatures(degree=10, include_bias=False)
X_poly = poly.fit_transform(X)
scaler = StandardScaler()
X_poly_scaled = scaler.fit_transform(X_poly)

# RidgeCV自动执行交叉验证
ridge_cv = RidgeCV(
    alphas=np.logspace(-3, 3, 100),  # 候选alpha值
    cv=5,                             # 5折交叉验证
    scoring='neg_mean_squared_error'  # 评估指标
)

ridge_cv.fit(X_poly_scaled, y.ravel())

print("RidgeCV结果:")
print(f"  最佳α: {ridge_cv.alpha_:.4f}")
print(f"  最佳得分: {ridge_cv.best_score_:.4f}")

---

## 🔄 第5部分：岭回归 vs 普通线性回归

让我们直观比较两者的区别：

In [ ]:
# ============================================================
# 岭回归 vs 普通线性回归
# ============================================================

# 创建两个模型
linear_model = Pipeline([
    ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
    ("scaler", StandardScaler()),
    ("linear_reg", LinearRegression())
])

ridge_model = Pipeline([
    ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
    ("scaler", StandardScaler()),
    ("ridge_reg", Ridge(alpha=best_alpha))
])

# 训练模型
linear_model.fit(X_train, y_train)
ridge_model.fit(X_train, y_train)

# 预测
y_train_linear = linear_model.predict(X_train)
y_test_linear = linear_model.predict(X_test)
y_line_linear = linear_model.predict(X_line)

y_train_ridge = ridge_model.predict(X_train)
y_test_ridge = ridge_model.predict(X_test)
y_line_ridge = ridge_model.predict(X_line)

# 真实曲线
y_true = TRUE_COEF_2 * X_line**2 + TRUE_COEF_1 * X_line + TRUE_COEF_0

# 可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# 左图：普通线性回归
axes[0].scatter(X_train, y_train, color='blue', alpha=0.6, s=50, label='训练集')
axes[0].scatter(X_test, y_test, color='green', alpha=0.3, s=30, label='测试集')
axes[0].plot(X_line, y_line_linear, 'r-', linewidth=2, label='预测曲线')
axes[0].plot(X_line, y_true, 'g--', linewidth=2, label='真实曲线')
axes[0].set_xlabel('X')
axes[0].set_ylabel('y')
axes[0].set_title(f'普通线性回归\n训练MSE: {mean_squared_error(y_train, y_train_linear):.2f}\n测试MSE: {mean_squared_error(y_test, y_test_linear):.2f}')
axes[0].set_ylim(-5, 15)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 中图：岭回归
axes[1].scatter(X_train, y_train, color='blue', alpha=0.6, s=50, label='训练集')
axes[1].scatter(X_test, y_test, color='green', alpha=0.3, s=30, label='测试集')
axes[1].plot(X_line, y_line_ridge, 'r-', linewidth=2, label='预测曲线')
axes[1].plot(X_line, y_true, 'g--', linewidth=2, label='真实曲线')
axes[1].set_xlabel('X')
axes[1].set_ylabel('y')
axes[1].set_title(f'岭回归 (α={best_alpha:.4f})\n训练MSE: {mean_squared_error(y_train, y_train_ridge):.2f}\n测试MSE: {mean_squared_error(y_test, y_test_ridge):.2f}')
axes[1].set_ylim(-5, 15)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# 右图：参数对比
linear_coefs = linear_model.named_steps['linear_reg'].coef_
ridge_coefs = ridge_model.named_steps['ridge_reg'].coef_.flatten()

x_pos = np.arange(len(linear_coefs))
width = 0.35

axes[2].bar(x_pos - width/2, np.abs(linear_coefs), width, label='线性回归', alpha=0.7)
axes[2].bar(x_pos + width/2, np.abs(ridge_coefs), width, label='岭回归', alpha=0.7)
axes[2].set_xlabel('参数索引')
axes[2].set_ylabel('|参数值|')
axes[2].set_title('参数绝对值对比')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 打印详细比较
print("\n" + "="*60)
print("模型性能比较")
print("="*60)
print(f"{'指标':<20} {'普通线性回归':>15} {'岭回归':>15}")
print("-"*60)
print(f"{'训练MSE':<20} {mean_squared_error(y_train, y_train_linear):>15.4f} {mean_squared_error(y_train, y_train_ridge):>15.4f}")
print(f"{'测试MSE':<20} {mean_squared_error(y_test, y_test_linear):>15.4f} {mean_squared_error(y_test, y_test_ridge):>15.4f}")
print(f"{'参数L2范数':<20} {np.linalg.norm(linear_coefs):>15.4f} {np.linalg.norm(ridge_coefs):>15.4f}")
print(f"{'最大参数值':<20} {np.max(np.abs(linear_coefs)):>15.4f} {np.max(np.abs(ridge_coefs)):>15.4f}")

---

## 📝 总结

### 关键要点

1. ✅ **岭回归**是带有L2正则化的线性回归
2. ✅ 通过惩罚大的参数值来防止过拟合
3. ✅ 正则化参数α控制正则化强度：
   - α过小：可能过拟合
   - α过大：可能欠拟合
4. ✅ 使用交叉验证选择最佳α值
5. ✅ 岭回归将参数收缩向0，但不会精确为0

### 岭回归 vs Lasso回归

| 特性 | 岭回归 (L2) | Lasso回归 (L1) |
|------|------------|----------------|
| 正则化项 | $\alpha\sum\theta_j^2$ | $\alpha\sum|\theta_j|$ |
| 参数收缩 | 收缩向0，不为0 | 可精确收缩到0 |
| 特征选择 | 不能 | 能 |
| 多重共线性 | 处理较好 | 倾向于选择一个 |

### 适用场景

- 特征数量较多
- 存在多重共线性
- 需要所有特征的模型
- 防止过拟合

### 注意事项

> ⚠️ **重要**: 使用岭回归前，需要对特征进行**标准化**，因为正则化对特征尺度敏感

> 💡 **建议**: 如果需要特征选择功能，考虑使用Lasso或Elastic Net

### 下一步学习

- 📘 Lasso回归 (L1正则化)
- 📙 Elastic Net (L1+L2)
- 📕 正则化超参数调优

### 练习题

1. 尝试使用不同的solver（'auto', 'svd', 'sparse_cg', 'lsqr', 'sag'），比较速度
2. 在真实数据集上应用岭回归，观察正则化效果
3. 比较岭回归和Lasso在特征选择上的差异

## 📚 参考资料

- [Scikit-learn: Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
- [Scikit-learn: RidgeCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)
- 《机器学习实战》第4章 - 正则化线性模型